## First step to clean the dataset:

- Select only English movies
- Select onlu full-length feature (movies of more than one hour long)
- Remove movies for which we don't have the title

In [2]:
import pandas as pd
#import requests
import numpy as np
import string
import json

In [3]:
character_metadata = pd.read_csv('Data/character.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_release_date',
                                                                              'Character_name',
                                                                              'Actor_date_of_birth',
                                                                              'Actor_gender',
                                                                              'Actor_height',
                                                                              'Actor_ethnicity',
                                                                              'Actor_name',
                                                                              'Actor_age_at_movie_release',
                                                                              'Freebase_character/actor_map_ID',
                                                                              'Freebase_character_ID',
                                                                              'Freebase_actor_ID'])

movie_metadata = pd.read_csv('Data/movie.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_name',
                                                                              'Movie_release_date',
                                                                              'Movie_box_office_revenue',
                                                                              'Movie_runtime',
                                                                              'Movie_languages',
                                                                              'Movie_countries',
                                                                              'Movie_genres'])

plot_summaries = pd.read_csv('Data/plot_summaries.txt', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'summary'])

In [140]:
#Copy of movie_metadata dataset
movie_metadata_filtered = movie_metadata.copy(deep= True)

#Filtering of movie_metada, keeping only full-length features movies (more than 60 minutes long), in English and with that have a Title in the database.
movie_metadata_filtered = movie_metadata_filtered[movie_metadata_filtered.Movie_languages.str.contains("English")]
movie_metadata_filtered.dropna(subset=['Movie_name'],inplace=True)
movie_metadata_filtered = movie_metadata_filtered[movie_metadata_filtered.Movie_runtime >= 60]

print(movie_metadata_filtered)

       Wikipedia_movie_ID Freebase_movie_ID  \
0                  975900         /m/03vyhn   
1                 3196793         /m/08yl5d   
3                 9363483        /m/0285_cd   
6                18998739        /m/04jcqvw   
7                10408933        /m/02qc0j7   
...                   ...               ...   
81735            32468537        /m/0crwd9y   
81736            35228177        /m/0j7hxnt   
81737            34980460        /m/0g4pl34   
81738             9971909        /m/02pygw1   
81740            12476867        /m/02w7zz8   

                                              Movie_name Movie_release_date  \
0                                         Ghosts of Mars         2001-08-24   
1      Getting Away with Murder: The JonBenét Ramsey ...         2000-02-16   
3                                       White Of The Eye               1987   
6                              The Sorcerer's Apprentice               2002   
7                               Alexande

In [5]:
movie_metadata_filtered.Movie_name[movie_metadata_filtered.Movie_name.str.contains(" la ")].head()

#There are still some Title that are not in English...

21500                              Murder a la Mod
33420    La pelota vasca. La piel contra la piedra
39681                             Café de la plage
40550                         Les Damnés de la mer
45773               Lumumba, la mort d'un prophète
Name: Movie_name, dtype: object

In [6]:
movie_English_only = movie_metadata_filtered[movie_metadata_filtered.Movie_languages == '{"/m/02h40lc": "English Language"}']

In [7]:
movie_English_only.Movie_name[movie_English_only.Movie_name.str.contains(" la ")].head()

21500    Murder a la Mod
70558      Irma la Douce
Name: Movie_name, dtype: object

In [8]:
movie_English_only.to_csv(path_or_buf = "Data/movies_metadata_english_only.csv", sep=',')

In [9]:
# IMDB files 

ratings_imdb = pd.read_table('Data/ratings_imdb.tsv', sep="\t")
ratings_imdb.head()


movies_imdb= pd.read_table('Data/movies_imdb.tsv', sep="\t")



In [10]:
# Genre processing
# Objectif: donner des noms de genre communs pour tous les films
#(grosse) Inspiration du projet adamax 2022 (spliter + les genres ou en faire moins?)

### 1st step: create a new genre dictionnary /!\ they did it themselves, should we do it as well?
dict_genre = {
    "Porn": ["Pornographic movie", "Pornography", "Softcore Porn", "Adult", "Sexploitation", "Erotica", "homoeroticism"], 
    "Action": ["Action/adventure", "ninja movie", "epic", "auto racing", "martial arts film", "race movie", "samurai cinema"],
    "Adventure": ["Adventure", "travel"],
    "Animals": ["Animal Picture"],
    "Anime": [],
    "Alien Film": ["Alien invasion"],
    "Animation": ["Animated cartoon"],
    "Beach Film": ["Beach Party film"],
    "Biographical film": ["Biography", "Biopic [feature]"],
    "Buddy film": ["Buddy Picture", "Buddy cop"],
    "Coming of age": ["Coming-of-age film"],
    "Comedy": ["Humour", "Tragicomedy", "Ealing Comedies", "Comdedy", "Adventure Comedy", "Commedy of manners", "Workplace comedy", "comedy-drama"], 
    "Crime": ['crime', 'crime fiction', 'crime thriller', "master criminal films", "detective"],
    "Documentary": ["Political Documetary", "Docudrama", "Historical Documentaries", "Rockumentary", "media studies", "media satire"], 
    "Drama": ["Melodrama", "Tragedy", "Tragicomedy", "Docudrama", "Family drama", "Comedy-drama", 'illnesses & disabilities'], 
    "Family": ["Family Film"],
    "Fiction": ["fairy tail", "fictional film", "medical fiction", "mythological fantasy", "sci fi pictures", 'sci fi pictures original films', "sword and sorcery films", "sword and sandal", "time travel", "supernatural"],
    "Gross out": ["Gross-out Film"],
    "Historical": ["history", "Biographical", "British History", "British Empire Film", "British new wave", "Historical Epic", "Cold War", "Historical Documentaries"],
    "Monster": ["Creature Movie", "Monster Movie", "Creature"],
    "Horror": ["Natural Horror film", "Sci-fi Horror", "road-horror"],
    "Indie": ["Indie"],
    "Language & Literature": ["& Literature"], 
    "LGBT": ["New Queer Cinema", "Gay", "Homoeroticism"],
    "Old movies": ["Silent film", "black-and-white"],
    "Parody": [],
    "Political cinema": ["Political Documentary"],
    "Prison": ["Prison film", "Women in prison film"],
    "Master Criminal Films": ["Heist", "Gangster Film"],
    "Music": ["Concert film", "Film-Opera", "Operetta", "Punk rock", "Rockumentary", "Space opera", "Singing cowboy", "Hip hop movies", "Breakdance", "Dance", "musical"],
    "For children": ["Children's film", "children's", "children's entertainment", "children's fantasy", "children's/family"],
    "Religious Film": ["Christian film", "Cult"],
    "Romance": ["Romance film", "romantic fantasy"],
    "Sport": ["baseball", "boxing", "sports"],
    "Super hero": ["superhero movie", "superhero"],
    "Social issues": ["Social problem film"], 
    "Television Movie": ["TV Movie"],
    "Thriller": ["erotic thriller", "film à clef", "mystery", "suspense"],
    "War film": ["war", "War effort", "The Netherlands in World War II", "Gulf War", "Combat Films", "movies about gladiators", "anti-war", "anti-war film", "nuclear warfare", "foreign legion"],
    "Western": ["Singing cowboy", "Westerns", "B-Western", "Epic Western", "Spaghetti Western"],
    "World cinema": ["Chinese Movies", "Japanese Movies", "Latino", "americana", "bengali cinema", "filipino movies"],
    "Zombie": ["Zombie film", "Z-movie", "z movie"]
}
## To discuss: "world cinema"--> keep or remove? diversity?


In [11]:
##2nd step: Revert the current dictionnary, easier to manipulate

dict_genre_revert = {}
for key, value in dict_genre.items():
    key = key.lower() #passage en minuscule
    for v in value:
        v = v.lower()
        dict_genre_revert[v] = key
    dict_genre_revert[key] = key



In [12]:
##3rd step= create a new set with the new genre

def get_genre(set_original_genres):
    set_new_genres = set()
    for g in set_original_genres:
        g = g.lower()
        temp = True
        for s_g in g.split(" "): 
            if s_g in dict_genre_revert.keys():
                set_new_genres.add(dict_genre_revert[s_g])
                temp = False
        if temp:
            if g in dict_genre_revert.keys():
                set_new_genres.add(dict_genre_revert[g])
    return set_new_genres

In [13]:
# Test process genre sur movie_English_only

df_test= movie_English_only.copy(deep=True)


In [14]:
## Rendre les colonnes lisibles (set de valeurs) -->intéressant pour d'autres colonnes!!
pd.set_option('mode.chained_assignment', None) #enlever les warnings de changement de valeur

for i in df_test.index:

    d=json.loads(df_test["Movie_genres"][i]) #transformer str en dict

    df_test["Movie_genres"][i]=set(d.values()) #retourne un set des valeurs du dict

### en faire une fonction?

In [21]:
## Changer les genres originaux en nouveux genres simplifiés
for i in df_test.index:
    df_test["Movie_genres"][i] = get_genre(df_test["Movie_genres"][i])

print(get_genre(df_test["Movie_genres"][1]))
print(movie_English_only["Movie_genres"][1])



{'drama', 'historical', 'crime', 'thriller'}
{"/m/02n4kr": "Mystery", "/m/03bxz7": "Biographical film", "/m/07s9rl0": "Drama", "/m/0hj3n01": "Crime Drama"}


# preprocess character metadata before to merge it with movies



In [136]:
# Ajouter les noms des personnages principaux au movie.metadata

character_metadata_copy= character_metadata.copy(deep= True)
character_metadata_copy.dropna(subset=['Character_name'], inplace=True)
#il reste encore des NaN !!!!

movie_character = movie_English_only.merge(character_metadata_copy, how='left', left_on='Wikipedia_movie_ID', right_on='Wikipedia_movie_ID')

movie_character = movie_character[['Wikipedia_movie_ID', 'Movie_name',  'Movie_box_office_revenue', 'Movie_runtime', 'Movie_languages', 'Movie_countries', 'Movie_genres', 'Character_name', 'Freebase_character_ID']]
# je sais pas pourquoi mais il veut pas mettre le wikipedia movie ID et la release date

grouped_movie_character = movie_character.groupby('Wikipedia_movie_ID').agg({
    'Movie_name': 'first',  
    'Movie_box_office_revenue': 'first', 
    'Movie_runtime': 'first', 
    'Movie_languages': 'first', 
    'Movie_countries': 'first',  
    'Movie_genres': 'first',  
    'Character_name': lambda x: ', '.join(x.dropna()),  
    'Freebase_character_ID':  lambda x: ', '.join(x.dropna())
}).reset_index()
grouped_movie_character.sample(5).head()

,Wikipedia_movie_ID,Movie_name,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Character_name,Freebase_character_ID
9801,6270685,Tiptoes,NaN,90.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/06cvj"": ""Romantic comedy"", ""/m/0219x_"": ""...","Carol, Steven Bedalia, Rolfe, Lucy, Maurice, S...","/m/0gxbc2b, /m/0ghtvx2, /m/0gy9v2l, /m/0h282wx..."
20207,23594997,Gulliver's Travels,237382724.0,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hqxf"": ""Family Film"", ""/m/0hj3n2k"": ""Fan...","Dan, Lemuel Gulliver, Horatio, Princess Mary, ...","/m/0g1__n3, /m/083m72x, /m/083m728, /m/08x2q7v..."
6452,3517536,The Lovely Bones,93621340.0,136.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/03q4nz"": ""World ...","Lindsey Salmon, Jack Salmon, Abigail Salmon, G...","/m/07yk3gx, /m/07yk3d_, /m/07yk3hh, /m/07yk3h7..."
6867,3782892,Pack Up Your Troubles,NaN,68.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01z4y"": ""Comedy"", ""/m/01g6gs"": ""Black-and...",,
11793,8752596,Dead Clever,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0vgkd"": ""Black c...","Daniel, Young Nathan, Gaynor, Judge, D.S. Liam...","/m/0kxpdyq, /m/0kxnd7d, /m/0kxlzgs, /m/0kxp3jj..."


In [137]:
# check if one of the character's name is present in the movie title

movie_character['Character_in_title'] = movie_character.apply(lambda row: str(row['Character_name']).lower() in str(row['Movie_name']).lower(), axis=1)
movie_with_character_in_title = movie_character['Character_in_title'].sum()


print(f"Fraction of movie titles containing character name: {movie_with_character_in_title / len(movie_character):.2%}")
print(movie_character.loc[movie_character['Character_in_title'] == 1, ['Movie_name', 'Character_name']])

Fraction of movie titles containing character name: 1.21%
                                     Movie_name   Character_name
29                                      Henry V          Henry V
38                                 Mary Poppins     Mary Poppins
200                 ChromeSkull: Laid to Rest 2      ChromeSkull
272                      Good Morning Miss Dove        Miss Dove
399                                   RoboCop 3          RoboCop
...                                         ...              ...
108887                             Stingray Sam     Stingray Sam
109242                                    Emile            Emile
109308                            Grandma's Boy    Grandma's Boy
109768  Sherlock Holmes and the Voice of Terror  Voice of Terror
109777  Sherlock Holmes and the Voice of Terror  Sherlock Holmes

[1326 rows x 2 columns]


In [ ]:
# certains noms de personnages ne sont aps vraiment des prénoms, est-ce qu'il faudra faire de la NLP?


In [145]:
# years corrected

movie_metadata_temporal = movie_metadata_filtered.copy(deep= True)
movie_metadata_temporal.dropna(subset=['Movie_release_date'], inplace = True)
movie_metadata_temporal["Movie_release_date"] = movie_metadata_temporal['Movie_release_date'].apply(lambda x : str(x)[:4]).apply(lambda x : int(x))
